In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
!nvidia-smi

Sat Apr  5 07:44:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
%%writefile vec-add.cu
#include <cassert>
#include <cstdlib>

__global__ void vecAddKernel(float *A, float *B, float *C, int n) {
  int i = threadIdx.x + blockDim.x * blockIdx.x;
  if (i < n) {
    C[i] = A[i] + B[i];
  }
}

void vecAdd(float *A, float *B, float *C, int n) {
  float *A_d, *B_d, *C_d;
  int size = n * sizeof(float);

  cudaMalloc((void **)&A_d, size);
  cudaMalloc((void **)&B_d, size);
  cudaMalloc((void **)&C_d, size);

  cudaMemcpy(A_d, A, size, cudaMemcpyHostToDevice);
  cudaMemcpy(B_d, B, size, cudaMemcpyHostToDevice);

  vecAddKernel<<<ceil(n / 256.0), 256>>>(A_d, B_d, C_d, n);

  cudaMemcpy(C, C_d, size, cudaMemcpyDeviceToHost);

  cudaFree(A_d);
  cudaFree(B_d);
  cudaFree(C_d);
}

int main() {
  int n = 128;
  size_t size = n * sizeof(float);
  float *A_h = (float *)malloc(size);
  float *B_h = (float *)malloc(size);
  float *C_h = (float *)malloc(size);

  for (size_t i = 0; i < n; i++) {
    A_h[i] = 1;
    B_h[i] = 2;
  }

  vecAdd(A_h, B_h, C_h, n);

  for (size_t i = 0; i < n; i++) {
    assert(C_h[i] == 3);
  }

  return 0;
}

Overwriting vec-add.cu


In [9]:
!nvcc -o vec-add vec-add.cu -arch=sm_75 -run

vec-add: vec-add.cu:46: int main(): Assertion `C_h[i] == 2' failed.
Aborted (core dumped)


In [10]:
!./vec-add

vec-add: vec-add.cu:46: int main(): Assertion `C_h[i] == 2' failed.
